In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_fbref_xG(url):
    
    # Creatind a dictionary of index and labels
    
    dict_index = { 
        2: '_Standard',
        4: '_GK',
        6: '_AdvGK',
        8: '_Shooting',
        10: '_Passing',
        12: '_PassTypes',
        14: 'G&SCreation',
        16: '_DefActions',
        18: '_Possession',
        20: '_PlayTime',
        22: '_Misc'
    }
    
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')
    
    # Initializing an empty dataframe
    merged_df = None
    
    for index, label in dict_index.items():
        
        # Extract the specific table (assuming it's the 3rd table on the page)
        tables = data.find_all('table')
        rows = tables[index].find_all('tr')

        # Extract the headers
        headers = rows[1].find_all('th')
        column_titles = [header.get_text(strip=True) for header in headers]

        # Creating title label for accompanying the default naming convention
        ## IF statements here
        title_label = label

        # Add '_Standard' to each column name
        column_titles = [column_titles[0]] + [title + title_label for title in column_titles[1:]]

        # Check the number of columns expected
        print(f"Expected number of columns: {len(column_titles)}")

        # Extract the data rows
        table_data = []
        for row in rows[1:]:  # Start from the second row to skip the header row
            cols_a = [col.get_text(strip=True) for col in row.find_all('a')]
            cols_b = [col.get_text(strip=True) for col in row.find_all('td')]
            combined_cols = cols_a + cols_b

            # Print the length of combined columns for debugging
            # print(f"Row has {len(combined_cols)} columns: {combined_cols}")

            # Append only if the number of columns matches the headers
            # The first row was empty, that's why the error in the previous block of code. Keep this one
            if len(combined_cols) == len(column_titles):
                table_data.append(combined_cols)
            else:
                print(f"Skipping row with {len(combined_cols)} columns, expected {len(column_titles)}.")
    
        # Create DataFrame and set column titles
        df = pd.DataFrame(table_data, columns=column_titles)
        
        # Merge dataframe with the existing dataframe
        if merged_df is None:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, how='outer', on='Squad')
            
    # Adding an extraction date column
    today = datetime.today()
    merged_df['Extraction Date'] = today
    
    return merged_df

# Example usage
url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
epl_df = scrape_fbref_xG(url)

epl_df

Expected number of columns: 32
Skipping row with 0 columns, expected 32.
Expected number of columns: 21
Skipping row with 0 columns, expected 21.
Expected number of columns: 28
Skipping row with 0 columns, expected 28.
Expected number of columns: 20
Skipping row with 0 columns, expected 20.
Expected number of columns: 26
Skipping row with 0 columns, expected 26.
Expected number of columns: 18
Skipping row with 0 columns, expected 18.
Expected number of columns: 19
Skipping row with 0 columns, expected 19.
Expected number of columns: 19
Skipping row with 0 columns, expected 19.
Expected number of columns: 26
Skipping row with 0 columns, expected 26.
Expected number of columns: 23
Skipping row with 0 columns, expected 23.
Expected number of columns: 19
Skipping row with 0 columns, expected 19.


,Squad,# Pl_Standard,Age_Standard,Poss_Standard,MP_Standard,Starts_Standard,Min_Standard,90s_Standard,Gls_Standard,Ast_Standard,...,Int_Misc,TklW_Misc,PKwon_Misc,PKcon_Misc,OG_Misc,Recov_Misc,Won_Misc,Lost_Misc,Won%_Misc,Extraction Date
0,Arsenal,23,26.4,48.7,9,99,810,9.0,16,13,...,67,100,0,1,0,321,125,122,50.6,2024-11-03 19:29:38.853584
1,Aston Villa,21,27.2,52.2,9,99,810,9.0,15,14,...,63,89,0,2,0,324,79,102,43.6,2024-11-03 19:29:38.853584
2,Bournemouth,23,25.7,44.7,9,99,810,9.0,11,9,...,78,95,2,0,0,463,118,144,45.0,2024-11-03 19:29:38.853584
3,Brentford,20,26.4,48.3,9,99,810,9.0,18,10,...,77,100,2,0,1,391,135,115,54.0,2024-11-03 19:29:38.853584
4,Brighton,26,26.2,55.7,9,99,810,9.0,16,11,...,77,88,0,2,0,400,119,104,53.4,2024-11-03 19:29:38.853584
5,Chelsea,23,23.7,55.1,9,99,810,9.0,19,16,...,80,89,1,2,0,387,81,90,47.4,2024-11-03 19:29:38.853584
6,Crystal Palace,23,26.4,47.0,9,99,810,9.0,5,3,...,70,116,1,0,0,416,130,170,43.3,2024-11-03 19:29:38.853584
7,Everton,22,29.0,38.4,9,99,810,9.0,10,7,...,95,110,0,1,0,405,147,117,55.7,2024-11-03 19:29:38.853584
8,Fulham,21,28.3,50.8,9,99,810,9.0,12,11,...,76,95,1,0,1,354,104,116,47.3,2024-11-03 19:29:38.853584
9,Ipswich Town,27,26.3,41.3,9,99,810,9.0,9,7,...,81,87,0,2,0,332,115,113,50.4,2024-11-03 19:29:38.853584


In [5]:
fixtures = pd.read_html('https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures')

In [13]:
fixtures_df = fixtures[0]
fixtures_df = fixtures_df[fixtures_df['Match Report'] == 'Head-to-Head']
fixtures_df.columns

Index(['Wk', 'Day', 'Date', 'Time', 'Home', 'xG', 'Score', 'xG.1', 'Away',
       'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'],
      dtype='object')

In [17]:
fixtures_df = fixtures_df[['Home', 'Away', 'Date']]

In [25]:
fixtures_df['Match'] = fixtures_df['Home'] +' ' + 'Vs' + ' ' +  fixtures_df['Away']

In [27]:
fixtures_df

,Home,Away,Date,Match
99,Newcastle Utd,Arsenal,2024-11-02,Newcastle Utd Vs Arsenal
100,Southampton,Everton,2024-11-02,Southampton Vs Everton
101,Liverpool,Brighton,2024-11-02,Liverpool Vs Brighton
102,Nott'ham Forest,West Ham,2024-11-02,Nott'ham Forest Vs West Ham
103,Ipswich Town,Leicester City,2024-11-02,Ipswich Town Vs Leicester City
...,...,...,...,...
412,Fulham,Manchester City,2025-05-25,Fulham Vs Manchester City
413,Nott'ham Forest,Chelsea,2025-05-25,Nott'ham Forest Vs Chelsea
414,Manchester Utd,Aston Villa,2025-05-25,Manchester Utd Vs Aston Villa
415,Wolves,Brentford,2025-05-25,Wolves Vs Brentford


In [35]:
merged_df = pd.merge(epl_df, fixtures_df, left_on='Squad', right_on='Home')

In [43]:
merged_df.to_csv('epl_df_merged')

In [47]:
merged_fixtures = pd.merge(fixtures_df, epl_df, left_on='Home', right_on='Squad')

In [49]:
merged_fixtures.to_csv('fixtures_merged')